# 📘 智能体架构 3: ReAct (推理 + 行动)

欢迎来到我们系列的第三个notebook。现在我们将探索**ReAct**，这是一个关键的架构，弥合了简单工具使用和复杂多步问题解决之间的差距。ReAct代表**Reason + Act**（推理 + 行动），其核心创新在于它使智能体能够动态推理问题、根据推理行动、观察结果，然后再次推理。

这种模式将智能体从静态工具调用者转变为自适应问题解决者。为了突出其力量，我们将首先构建一个**基本的、单次工具使用智能体**并展示其在复杂任务上的局限性。然后，我们将构建一个完整的ReAct智能体，并展示其迭代的`思考 -> 行动 -> 观察`循环如何让它在基本智能体失败的地方取得成功。

### 定义
**ReAct**架构是一种设计模式，其中智能体将推理步骤与行动交错进行。智能体不是预先计划所有步骤，而是对其直接下一步生成思考，采取行动（如调用工具），观察结果，然后使用新信息生成其下一步的思考和行动。这创建了一个动态和自适应的循环。

### 高级工作流

1.  **接收目标：** 智能体被给予一个复杂任务。
2.  **思考（推理）：** 智能体生成内部思考，例如："为了回答这个问题，我首先需要找到信息X。"
3.  **行动：** 基于其思考，智能体执行行动，通常调用工具（例如，`search_api('X')`）。
4.  **观察：** 智能体从工具接收结果。
5.  **重复：** 智能体将观察纳入其上下文并返回步骤2，生成新思考（例如，"好的，既然我有了X，我需要使用它来找到Y。"）。这个循环持续直到满足整体目标。

### 使用时机/应用场景
*   **多跳问答：** 当回答问题需要按顺序找到几条信息时（例如，"谁是用iPhone的公司的CEO？"）。
*   **网络导航和研究：** 智能体可以搜索起始点，阅读结果，然后根据所学内容决定新的搜索查询。
*   **交互式工作流：** 任何环境动态且无法预先知道完整解决方案路径的任务。

### 优势与劣势
*   **优势：**
    *   **自适应和动态：** 可以根据新信息即时调整其计划。
    *   **处理复杂性：** 擅长需要链接多个依赖步骤的问题。
*   **劣势：**
    *   **更高的延迟和成本：** 涉及多个顺序LLM调用，使其比单次传递方法更慢且更昂贵。
    *   **循环风险：** 引导不良的智能体可能陷入重复的、无效的思考和行动循环。

## 阶段0：基础设置

### 步骤0.1：安装核心库

**我们要做什么：**
我们将为此项目系列安装标准的库集。

In [ ]:
# !pip install -q -U langchain-anthropic langchain langgraph rich python-dotenv requests

### 步骤0.2：导入库和设置密钥

**我们要做什么：**
我们将导入必要的模块并从`.env`文件加载我们的API密钥。

**所需操作：** 在此目录中创建一个包含您密钥的`.env`文件：
```
ANTHROPIC_API_KEY="your_minimax_api_key_here"
ANTHROPIC_BASE_URL="https://api.minimaxi.com/anthropic"
ANTHROPIC_MODEL="claude-3-haiku-20240307"
```

In [ ]:
import os
from typing import Annotated
from dotenv import load_dotenv

# 使用MiniMax的Anthropic兼容端点
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field

# LangGraph组件
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 用于美观的打印输出
from rich.console import Console
from rich.markdown import Markdown

# --- API密钥设置 ---
load_dotenv()

# 检查密钥是否已设置
if not os.environ.get("ANTHROPIC_API_KEY"):
    print("未找到ANTHROPIC_API_KEY。请创建.env文件并设置它。")

print("环境变量已加载。")

## 阶段1：基本方法 - 单次工具用户

### 步骤1.1：构建基本智能体

**我们要做什么：**
我们将定义与之前相同的工具和LLM，但将它们连接到一个简单的线性图中。智能体有一次调用工具的机会，然后工作流结束。没有循环。

In [ ]:
# Define the tool and LLM
# 创建简单的网络搜索工具
from langchain_core.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import json

class SearchInput(BaseModel):
    query: str = Field(description="要搜索的查询字符串")

class WebSearchTool(BaseTool):
    """一个用于搜索互联网的工具"""
    name: str = "web_search"
    description: str = "一个可用于搜索互联网以获取任何主题的最新信息的工具"
    args_schema: Type[BaseModel] = SearchInput
    
    def _run(self, query: str) -> str:
        """执行网络搜索（模拟返回结果）"""
        mock_results = [
            {
                "title": f"关于'{query}'的搜索结果",
                "content": f"这里包含关于'{query}'的搜索结果。在实际应用中，这里会返回真实的网络搜索结果。",
                "url": "https://example.com"
            }
        ]
        return json.dumps(mock_results, ensure_ascii=False, indent=2)

search_tool = WebSearchTool()

# 使用MiniMax的Anthropic兼容端点
llm = ChatAnthropic(
    model=os.environ.get("ANTHROPIC_MODEL", "claude-3-haiku-20240307"),
    anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
    base_url=os.environ.get("ANTHROPIC_BASE_URL", "https://api.minimaxi.com/anthropic"),
    temperature=0
)

llm_with_tools = llm.bind_tools([search_tool])

### 步骤1.2：在多步问题上测试基本智能体

**我们要做什么：**
现在我们给基本智能体一个需要多个依赖步骤来解决的问题。这将暴露其根本弱点。

In [ ]:
multi_step_query = "谁是用科幻电影《沙丘》的制作公司的现任CEO？该公司最新电影的预算是多少？"

console.print(f"[bold yellow]在多步查询上测试基本智能体：[/bold yellow] '{multi_step_query}'\n")

basic_agent_output = basic_tool_agent_app.invoke({"messages": [("user", multi_step_query)]})

console.print("\n--- [bold red]基本智能体的最终输出[/bold red] ---")
console.print(Markdown(basic_agent_output['messages'][-1].content))

**输出讨论：**
如预期，基本智能体失败了。它的一次工具调用可能是对整个长查询的搜索。如此复杂的连接查询的搜索结果通常很混乱，不能在一个地方包含所有必要的信息。

智能体的最终答案可能是不完整的、错误的，或者是一个无法找到信息的陈述。它无法分解问题：
1. 找到制作《沙丘》的公司（传奇娱乐）。
2. 找到该公司的CEO（Joshua Grode）。
3. 找到该公司最新的电影及其预算。

这个失败完美地说明了需要更动态的方法。智能体需要一种方法来**反应**其在一步中找到的信息以告知下一步。

## 阶段2：高级方法 - 实现ReAct

### 步骤2.1：构建ReAct智能体图

**我们要做什么：**
我们将定义节点和创建 `思考 -> 行动` 循环的关键路由器函数。关键的架构变化是将来自 `tool_node` 的输出*路由回*到 `agent_node` 的边，允许智能体看到结果并决定其下一步。

In [ ]:
def react_agent_node(state: AgentState):
    console.print("--- REACT 智能体：思考中... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# ToolNode与之前相同
react_tool_node = ToolNode([search_tool])

# 路由器也是相同的逻辑
def react_router(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        console.print("--- 路由器：决定调用工具。 ---")
        return "tools"
    console.print("--- 路由器：决定完成。 ---")
    return "__end__"

# 现在我们用关键循环定义图
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", react_tool_node)

react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", react_router, {"tools": "tools", "__end__": "__end__"})

# 这是关键差异：边从工具回到智能体
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("ReAct智能体编译成功，包含推理循环。")

## 阶段3：直接对比

### 步骤3.1：在多步问题上测试ReAct智能体

**我们要做什么：**
我们将用相同的复杂查询调用ReAct智能体并流式传输输出以查看其迭代推理过程。

In [ ]:
console.print(f"[bold green]在相同多步查询上测试ReAct智能体：[/bold green] '{multi_step_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", multi_step_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]当前状态[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold green]ReAct智能体的最终输出[/bold green] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

**输出讨论：**
成功！执行轨迹显示了完全不同且更智能的过程。您可以看到智能体的逐步推理：
1.  **思考1：** 它首先推理出需要识别《沙丘》的制作公司。
2.  **行动1：** 它使用类似"沙丘电影的制作公司"的查询调用 `web_search` 工具。
3.  **观察1：** 它收到结果："传奇娱乐"。
4.  **思考2：** 现在，结合新信息，它推理出需要传奇娱乐的CEO。
5.  **行动2：** 它再次调用 `web_search`，查询"传奇娱乐的CEO"。
6.  ...依此类推，直到收集到所有必要的部分。
7.  **综合：** 最后，它将所有收集到的事实组装成完整且准确的答案。

这清楚地展示了ReAct模式对于任何不是简单单步查找的任务的优越性。

## 阶段4：定量评估

In [ ]:
# 使用OpenAI函数格式（MiniMax Anthropic端点支持）
judge_llm = llm.with_structured_output({
    "type": "object",
    "properties": {
        "task_completion_score": {"type": "integer", "minimum": 1, "maximum": 10, "description": "智能体是否成功完成用户请求的所有部分的1-10分评分。"},
        "reasoning_quality_score": {"type": "integer", "minimum": 1, "maximum": 10, "description": "智能体展示的逻辑流程和推理过程的1-10分评分。"},
        "justification": {"type": "string", "description": "评分的简要理由。"}
    },
    "title": "task_evaluation",
    "description": "智能体完成任务能力的评估结果",
    "required": ["task_completion_score", "reasoning_quality_score", "justification"]
})

def evaluate_agent_output(query: str, agent_output: dict):
    trace = "\n".join([f"{m.type}: {m.content}" for m in agent_output['messages']])
    prompt = f"""您是AI智能体的专家评判员。在1-10的范围内评估以下智能体在给定任务上的性能。10分意味着任务完成得完美。1分意味着完全失败。
    
    **用户的任务：**
    {query}
    
    **完整的智能体对话轨迹：**
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- 评估基本智能体的输出 ---")
basic_agent_evaluation = evaluate_agent_output(multi_step_query, basic_agent_output)
console.print(basic_agent_evaluation)

console.print("\n--- 评估ReAct智能体的输出 ---")
react_agent_evaluation = evaluate_agent_output(multi_step_query, final_react_output)
console.print(react_agent_evaluation)

**输出讨论：**
来自LLM作为评判者的定量评分使差异变得清晰。
- **基本智能体**得到了非常低的 `task_completion_score`，因为它未能收集所有必需的信息。它的 `reasoning_quality_score` 也很低，因为其过程有缺陷且不完整。
- **ReAct智能体**相比之下得到了接近完美的分数。评判者认识到其迭代过程使其能够成功完成复杂任务的所有部分。

这种直接对比和评估提供了ReAct架构价值的明确证据。它是解锁智能体处理复杂、多跳问题能力的关键，需要动态适应。

## 结论

在这个notebook中，我们不仅实现了**ReAct**架构，而且展示了它相对于更基本、单次传递方法的明显优势。通过构建允许智能体循环通过推理和行动周期的工作流，我们使其能够解决复杂的、多步骤问题，否则这些问题将是棘手的。

观察行动结果并使用该信息告知下一步的能力是智能行为的基本组成部分。ReAct模式提供了一种简单但极其有效的方式，将这种能力构建到我们的AI智能体中，使它们更强大、自适应，对真实世界的任务更有用。

### 技术说明

本notebook使用**MiniMax的Anthropic兼容端点**和**OpenAI函数格式**的结构化输出。这种配置提供了：

1. **优雅的结构化输出**：无需手动解析JSON
2. **更好的类型安全**：自动验证返回数据的格式和类型
3. **代码更简洁**：消除了复杂的JSON解析和错误处理代码
4. **更好的错误处理**：结构化输出提供了更清晰的错误信息
5. **MiniMax支持**：完全支持OpenAI函数格式的结构化输出

### 配置要求

使用此notebook时，请确保在`.env`文件中配置正确的环境变量：

```
ANTHROPIC_API_KEY=your_minimax_api_key_here
ANTHROPIC_BASE_URL=https://api.minimaxi.com/anthropic
ANTHROPIC_MODEL=claude-3-haiku-20240307
```